## Extract Titles and links

**here is the links off sites that contain recycling projects.**

In [ ]:
urls=['https://www.instructables.com/living/gardening/projects/',
      'https://www.instructables.com/search/?q=recycle+Glass&projects=all',
      'https://www.instructables.com/search/?q=recycle+carton&projects=all',
      'https://www.instructables.com/search/?q=recycle+Electronics&projects=all',
      'https://www.instructables.com/search/?q=recycle+Metal&projects=all',
      'https://www.instructables.com/craft/printmaking/projects/?channel=leather&channel=mason-jars&channel=no-sew&channel=paper',
      'https://www.instructables.com/craft/cardboard/projects/?channel=cards&channel=reuse&channel=wallets',
      'https://www.instructables.com/search/?q=recycle+Plastic&projects=all',
      'https://www.instructables.com/search/?q=recycling&projects=all',
      'https://www.instructables.com/search/?q=how+to+recycle&projects=all',
      'https://www.instructables.com/search/?q=bottles&projects=all']

In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [112]:
driver_path = r"C:\Users\hp\Downloads\chromedriver-win64\chromedriver.exe"  # Replace with your ChromeDriver path
service = Service(driver_path)

# Initialize the WebDriver
driver = webdriver.Chrome(service=service)

# URL of the website
url = "https://www.instructables.com/search/?q=bottles&projects=all"
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

# Step 1: Click the "Load All" button
try:
    load_all_button = driver.find_element(By.CLASS_NAME, "_button_156n2_17._loadAll_18rgo_1")
    load_all_button.click()
    time.sleep(3)  # Wait for the action to complete
except Exception as e:
    print("Failed to find or click the 'Load All' button.")
    driver.quit()
    raise

# Step 2: Scroll to load articles dynamically
desired_count = 6000  # Number of articles to scrape
articles = []  # List to store unique article links
titles = []  # List to store corresponding article titles
scroll_pause_time = 3  # Time to pause after each scroll
scroll_height = driver.execute_script("return document.body.scrollHeight")

while len(articles) < desired_count:
    # Parse the current page source
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Find all articles on the page
    all_articles = soup.find_all("div", class_="_ibleCard_1qrfl_24")
    
    # Extract links and titles from the articles
    for article in all_articles:
        link_tag = article.find("a")
        if link_tag:
            article_link = "https://www.instructables.com" + link_tag.attrs["href"]
            article_title = article.find("a", class_="_title_1qrfl_47")  # Title class
            if article_link not in articles and article_title:
                articles.append(article_link)  # Append link to list
                titles.append(article_title.get_text().strip())  # Append title to list

    # Scroll down to load more content
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)  # Wait for new content to load
    
    # Check if scrolling has reached the bottom
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    if new_scroll_height == scroll_height:
        print("Reached the end of the page. No more articles to load.")
        break
    scroll_height = new_scroll_height

# Limit the results to the desired number of links
articles = articles[:desired_count]
titles = titles[:desired_count]

# Print the collected titles and links
print(f"Collected {len(articles)} articles:")
# Close the browser
driver.quit()


Reached the end of the page. No more articles to load.
Collected 5220 articles:


### Transform it into DataFrame

In [117]:
# Create a DataFrame to store the results
data= pd.DataFrame({
    'Title': titles,
    'Link': articles
})

# Print the DataFrame
data.head()

,Title,Link
0,Eco-Art: Transforming Bottles Into Planters,https://www.instructables.com/Eco-Art-Transfor...
1,Fake Body Made Out of Plastic Bottles.,https://www.instructables.com/Fake-Body-Made-O...
2,PETFusion Machine: Turn Waste PET Bottles Into...,https://www.instructables.com/PETFusion-Machin...
3,Smart Plastic Bottles Waste-Management System,https://www.instructables.com/Smart-Plastic-Bo...
4,helping slouchy boots with old water bottles,https://www.instructables.com/helping-slouchy-...


### Save data

In [118]:
data.to_csv('bottles.csv', index=False)

## Concatenat Data and remove duplicate links

In [152]:
recycling=pd.read_csv("recycling.csv")
recycleplastic=pd.read_csv("recycleplastic.csv")
recyclemetal=pd.read_csv("recyclemetal.csv")
recycleglass=pd.read_csv("recycleglass.csv")
recycleelectronics=pd.read_csv("recycleelectronics.csv")
recyclecarton=pd.read_csv("recyclecarton.csv")
living_gardening=pd.read_csv("living_gardening.csv")
how_to_recycle=pd.read_csv("how_to_recycle.csv")
craft2=pd.read_csv("craft2.csv")
craft1=pd.read_csv("craft1.csv")
bottles=pd.read_csv("bottles.csv")


In [155]:
linksdataset = pd.concat([recycling, recycleplastic, recyclemetal,recycleglass,recycleelectronics,recyclecarton,living_gardening,how_to_recycle,craft2,craft1,bottles], ignore_index=True)

In [156]:
linksdataset.shape

(24511, 2)

In [157]:
linksdataset.head()

,Title,Link
0,Cardboard and Waste Recycling: 3 Genius Recycl...,https://www.instructables.com/Cardboard-and-Wa...
1,Recycling PET Plastic to Filament,https://www.instructables.com/Recycling-PET-Pl...
2,Recycling Bin for Kids Made From Recycled Card...,https://www.instructables.com/Recycling-Bin-fo...
3,Cardboard to Plant Pots by Recycling Waste Pap...,https://www.instructables.com/Cardboard-to-Pla...
4,System of Recycling Trash (SORT),https://www.instructables.com/System-of-Recycl...


In [158]:
linksdataset['Link'].duplicated().sum()

4643

In [159]:
linksdataset_no_duplicates_no_duplicates = linksdataset.drop_duplicates()

In [160]:
linksdataset_no_duplicates

,Title,Link
0,Cardboard and Waste Recycling: 3 Genius Recycl...,https://www.instructables.com/Cardboard-and-Wa...
1,Recycling PET Plastic to Filament,https://www.instructables.com/Recycling-PET-Pl...
2,Recycling Bin for Kids Made From Recycled Card...,https://www.instructables.com/Recycling-Bin-fo...
3,Cardboard to Plant Pots by Recycling Waste Pap...,https://www.instructables.com/Cardboard-to-Pla...
4,System of Recycling Trash (SORT),https://www.instructables.com/System-of-Recycl...
...,...,...
24506,Rocket Launch Lamp,https://www.instructables.com/Rocket-Launch-Lamp
24507,Seltzer Coke Bottle,https://www.instructables.com/Seltzer-Coke-Bottle
24508,Post-apocalyptic Plastic Bottle Club,https://www.instructables.com/Post-apocalyptic...
24509,Suzuki Marauder GZ125,https://www.instructables.com/Suzuki-Marauder-...


In [161]:
linksdataset_no_duplicates.shape

(19868, 2)

## save data of titles and links of each article project.

In [151]:
linksdataset_no_duplicates.to_csv('linksdatset.csv', index=False,encoding='utf-8')